In [3]:
import pandas as pd


from NFLVersReader.src.nflverse_clean.utils import assert_not_null, assert_and_alert
from configs import get_config

import warnings

warnings.filterwarnings('ignore')

## Player Stats
These stats are critical to determining the state of a team

In [6]:
from NFLVersReader.src.nflverse_clean.nflverse_reader import load_pbp_participation
from NFLVersReader.src.nflverse_clean.database_loader import DatabaseLoader

participation_df = pd.read_csv("../../output/playbyplay2021_participation.csv", header=0)
participation_df.shape


(50368, 14)

In [7]:
participation_df.head()

,nflverse_game_id,old_game_id,play_id,possession_team,offense_formation,offense_personnel,defenders_in_box,defense_personnel,number_of_pass_rushers,players_on_play,offense_players,defense_players,n_offense,n_defense
0,2021_01_DAL_TB,2021090900,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
1,2021_01_DAL_TB,2021090900,40,DAL,NaN,NaN,NaN,NaN,NaN,43360;44962;53605;45063;46088;53544;46185;4669...,00-0033083;00-0033512;00-0034674;00-0036882;00...,00-0036643;00-0033595;00-0034515;00-0032637;00...,11,11
2,2021_01_DAL_TB,2021090900,55,TB,SHOTGUN,"1 RB, 1 TE, 3 WR",7.0,"4 DL, 2 LB, 5 DB",NaN,44896;53504;53441;44962;42403;42404;52421;2551...,00-0033921;00-0032243;00-0036406;00-0019596;00...,00-0036942;00-0036932;00-0033512;00-0032197;00...,11,11
3,2021_01_DAL_TB,2021090900,76,TB,SINGLEBACK,"1 RB, 2 TE, 2 WR",7.0,"4 DL, 2 LB, 5 DB",NaN,44896;53504;44962;42403;42404;52421;25511;4608...,00-0033921;00-0032243;00-0036406;00-0019596;00...,00-0036942;00-0033512;00-0032197;00-0034674;00...,11,11
4,2021_01_DAL_TB,2021090900,97,TB,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,"4 DL, 2 LB, 5 DB",5.0,44896;53504;53441;44962;42403;42404;52421;2551...,00-0033921;00-0032243;00-0036406;00-0019596;00...,00-0036942;00-0036932;00-0033512;00-0032197;00...,11,11


In [4]:
pbp_df = pd.read_csv("../../output/playbyplay_2021.csv", low_memory=False)
pbp_df.head()

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,...,out_of_bounds,home_opening_kickoff,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,xpass,pass_oe
0,1,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,NaN,NaN,NaN,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,40,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,55,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,-1.399805,NaN,NaN,NaN,NaN,NaN,0.491433,-49.143299
3,76,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,0.032412,1.165133,5.803177,4.0,0.896654,0.125098,0.697346,30.265415
4,100,2021_01_ARI_TEN,2021091207,TEN,ARI,REG,1,TEN,home,ARI,...,0,1,-1.532898,0.256036,4.147637,2.0,0.965009,0.965009,0.978253,2.174652


## playByPlay + particicpation

In [5]:
print(participation_df.shape)
print(pbp_df.shape)

(50368, 15)
(50712, 372)


In [6]:
from NFLVersReader.src.nflverse_clean.clean_player_stats import rename_pbp_columns, rename_pbp_participant_columns

rename_pbp_columns(pbp_df)
rename_pbp_participant_columns(participation_df)

In [7]:
from NFLVersReader.src.nflverse_clean.clean_player_stats import reconcile_join_keys



Missing participants:   fixed 176,  not fixed: 344
both          50368
left_only       344
right_only        0
Name: _merge, dtype: int64


In [8]:
# pbp to participation is a 1 to 1 join
#  the number of rows in the merged db
merged_df = pd.merge(pbp_df, participation_df, left_on='play_id', right_on='play_id', how='outer', indicator=True, suffixes=('_new', '_prev'))
merged_df = merged_df.copy()  # to cleanup the frame
merged_df['_merge'].value_counts()

both          50368
left_only       344
right_only        0
Name: _merge, dtype: int64

In [10]:
from NFLVersReader.src.nflverse_clean.utils import publish_warning

msg = str(merged_df.loc[(merged_df['_merge'] == 'left_only'), ['game_id_new']].value_counts())
publish_warning(msg, 1)

WARNING (1: game_id_new    
2021_17_NYG_CHI    173
2021_06_GB_CHI     171
dtype: int64


## EXPLODE player arrays a new play_id from game_id

In [4]:
offense_players = explode_column(participation_df, 'play_id', 'offense_players')
offense_players.head()

,play_id,offense_players
0,1,None
1,40,00-0033083
1,40,00-0033512
1,40,00-0034674
1,40,00-0036882


In [5]:
defense_players = explode_column(participation_df, 'play_id', 'defense_players')
defense_players.head()

,play_id,defense_players
0,1,None
1,40,00-0036643
1,40,00-0033595
1,40,00-0034515
1,40,00-0032637


In [6]:
pbp_players = explode_column(participation_df, 'play_id', 'players_on_play')
pbp_players.head()

,play_id,players_on_play
0,1,None
1,40,43360
1,40,44962
1,40,53605
1,40,45063
